# Topic Modeling with Gensim

A **topic model** is an abstraction of the major topics contained in a corpus of texts. "Topic" in this context simply means a pattern of co-occurring words. The assumption is that if there are clearly identified patterns of co-occurring words, those patterns of co-occurring words reveal a latent structure in the corpus of texts. In short, a topic model is a representation of the major themes or structures of a corpus of texts.

`Gensim` is a popular Python library for building topic models. In this notebook we will use `Gensim` to build a topic model of Gibbon's _Decline and Fall of the Roman Empire_. After building a topic model, we will then use `pyLDAvis` to visualize the model so we can evaluate its usefulness.

I highly recommend that you read through `Gensim`'s [documentation](https://radimrehurek.com/gensim/auto_examples/index.html#core-tutorials-new-users-start-here). Much of the code below is adapted from that source.

## Set up

**NOTE**: one of the Python libraries we are using (`pyLDAvis`) can cause problems. Be sure to do the installations in the order that you see them below.

In [1]:
! pip install funcy

In [2]:
! pip install tzdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 5.7 MB/s eta 0:00:00


In [3]:
! pip install --no-dependencies pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.3 MB/s eta 0:00:00


In [4]:
! pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=96c84b039d21b1d25a71b8effbc53af4c8f0f4f94f2bb0967a6bbc376e600743
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [9]:
from collections import defaultdict
import wget
from gensim import corpora, models
from google.colab import files  # <-- only necessary if you are using Colab
import io # <-- only necessary if you are using Colab
import pandas as pd
import pyLDAvis.gensim
import warnings
import requests
import spacy
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Upload data

### Class example

In [10]:
url = 'https://raw.githubusercontent.com/msaxton/nlp-data/main/gibbon_sections.csv'
file_name = wget.download(url)
df = pd.read_csv(file_name)
df.head()

,Unnamed: 0,doc_id,text,lemmas
0,0,01-0,The extent and military force of the Roman emp...,extent force empire century comprehend part ea...
1,1,01-1,"exalted situation, had much less to hope than ...",exalt situation hope fear chance arm prosecuti...
2,2,01-2,"and towards the south, the sandy deserts of A...",south desert imitate successor repose mankind ...
3,3,01-3,the love of freedom without the spirit of unio...,love freedom spirit union take arm fierceness ...
4,4,01-4,"line of military stations, which was afterwar...",line station fortify reign rampart erect found...


In [12]:
# set up nlp pipline
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')

['ner', 'parser']

In [13]:
# Pride and Prejudice - Jane Austen
response = requests.get('https://www.gutenberg.org/cache/epub/1342/pg1342.txt')
text = response.text

Isolate the test we are interested in

In [22]:
start = text.find('Chapter I.]')
start

35869

In [20]:
end = text.find('*** END OF THE PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***')
end

744217

In [24]:
data = text[start:end]

In [25]:
data_paras = data.split('\r\n\r\n')


In [27]:
# creating empty lists for author and title will be handy for building our dataframe
author = []
title = []
# this little for-loop will poplate the lists we just created
for para in data_paras:
    author.append('Austen')
    title.append('Pride')

In [28]:
data_df = pd.DataFrame(list(zip(author, title, data_paras)), columns=['author', 'title', 'text'])

In [29]:
# extract lemmas
def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    lemmas = [token.lemma_ for token in no_punct]
    lemmas_lower = [lemma.lower() for lemma in lemmas]
    lemmas_string = ' '.join(lemmas_lower)
    return lemmas_string

In [31]:
data_df['lemmas'] = data_df['text'].apply(process_text)
data_df.head()

,author,title,text,lemmas
0,Austen,Pride,Chapter I.],chapter
1,Austen,Pride,"\r\nIt is a truth universally acknowledged, th...",truth universally acknowledge single man posse...
2,Austen,Pride,However little known the feelings or views of ...,little know feeling view man enter neighbourho...
3,Austen,Pride,"“My dear Mr. Bennet,” said his lady to him one...",dear bennet say lady day hear netherfield park...
4,Austen,Pride,Mr. Bennet replied that he had not.,bennet reply


In [35]:
length_filter = data_df['lemmas'].str.len() > 25
filter_df = data_df[length_filter]

In [36]:
# remove \n and \r characters from the text
def remove_new_lines(text):
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    return text

filter_df['text'] = filter_df['text'].apply(remove_new_lines)

<ipython-input-36-8117912e3ed4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df['text'] = filter_df['text'].apply(remove_new_lines)


In [37]:
filter_df = filter_df.reset_index(drop=True)
filter_df

,author,title,text,lemmas
0,Austen,Pride,"It is a truth universally acknowledged, that...",truth universally acknowledge single man posse...
1,Austen,Pride,However little known the feelings or views of ...,little know feeling view man enter neighbourho...
2,Austen,Pride,"“My dear Mr. Bennet,” said his lady to him one...",dear bennet say lady day hear netherfield park...
3,Austen,Pride,“Do not you want to know who has taken it?” cr...,want know take cry wife impatiently
4,Austen,Pride,"“Why, my dear, you must know, Mrs. Long says t...",dear know long say netherfield take young man ...
...,...,...,...,...
1804,Austen,Pride,Miss Bingley was very deeply mortified by Darc...,miss bingley deeply mortify darcy marriage thi...
1805,Austen,Pride,Pemberley was now Georgiana’s home; and the at...,pemberley georgiana home attachment sister exa...
1806,Austen,Pride,Lady Catherine was extremely indignant on the ...,lady catherine extremely indignant marriage ne...
1807,Austen,Pride,With the Gardiners they were always on the mos...,gardiner intimate term darcy elizabeth love se...


## Prepare data for topic model
The Python library we are going to use to make our topic model requires the data to be in a form of a list. Within that list, each "document" is also a list. So it looks something like this:

`[
  ['This is document 1'],
  ['This is document 2'],
  ['This is document 3']
]`

In [38]:
# extract the data out of the DataFrame
documents = df['lemmas'].to_list()

`Gensim` needs each document to be tokenized. We can use [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp) to quickly achieve this result. When complete, our data will now look like this:

`[
  ['This', 'is', 'document', '1'],
  ['This', 'is', 'document', '2'],
  ['This', 'is', 'document', '3'],
]`

In [39]:
# tokenize - the syntax below will create a list of lists
texts =[
    [word for word in document.lower().split()]
    for document in documents
]

It takes a lot of preparation to build a useful topic model. An important part of that preparation is to eliminate "noise" from you model. One way to do this is to remove pieces of data that are irrelevant. Here we will remove tokens that only occur once. **You may want to adjust this as you refine your topic model.**

In [40]:
# create a count of each token
frequency = defaultdict(int)
for text in texts:
  for token in text:
    frequency[token] += 1

In [41]:
# remove words that appear only 1 time
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

## Build topic model

`Gensim` is built around [four core concepts](https://radimrehurek.com/gensim/auto_examples/core/run_core_concepts.html#core-concepts):
- **document**: an individual text. In our case, this is an individual section from Gibbon.
- **corpus**: a collection of documents. In our case, this is all the sections from Gibbon put together.
- **vector**: a mathematically convenient representation of a document. Basically, each word in the document is given a numerical id. This allows `Gensim` to do faster calculations behind the scene.
- **model**: an algorithm for transforming vectors from one representation to another. In our case, this will be the LDA model we build.


### Basic topic model



In [42]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
dictionary = corpora.Dictionary(texts)

In [43]:
# create a corpus based off our dictionary and our texts
corpus = [dictionary.doc2bow(text) for text in texts]

In [44]:
# build LDA model
lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20, passes=50)

In [45]:
# explore topics
lda_model.print_topics()

[(0,
  '0.009*"city" + 0.008*"church" + 0.006*"temple" + 0.005*"mind" + 0.005*"year" + 0.005*"silver" + 0.005*"empire" + 0.005*"treasure" + 0.005*"restore" + 0.005*"prophet"'),
 (1,
  '0.017*"conclave" + 0.010*"courage" + 0.010*"arch" + 0.009*"day" + 0.009*"pope" + 0.009*"ruin" + 0.008*"reign" + 0.008*"taste" + 0.008*"cardinal" + 0.007*"cause"'),
 (2,
  '0.011*"empire" + 0.008*"emperor" + 0.007*"province" + 0.007*"power" + 0.007*"name" + 0.007*"prince" + 0.006*"reign" + 0.006*"year" + 0.005*"nation" + 0.005*"form"'),
 (3,
  '0.010*"king" + 0.008*"emperor" + 0.008*"arm" + 0.008*"army" + 0.008*"city" + 0.007*"enemy" + 0.006*"conquest" + 0.005*"empire" + 0.005*"year" + 0.005*"prince"'),
 (4,
  '0.010*"city" + 0.009*"pope" + 0.007*"government" + 0.007*"people" + 0.007*"name" + 0.007*"state" + 0.007*"time" + 0.007*"century" + 0.006*"country" + 0.006*"character"'),
 (5,
  '0.010*"people" + 0.009*"city" + 0.008*"senator" + 0.007*"death" + 0.006*"name" + 0.006*"stone" + 0.005*"multitude" + 0.0

In [46]:
# Find topics in each document
lda_model.get_document_topics(corpus[0])

[(2, 0.92486286), (4, 0.06545812)]

In [47]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.038353 -0.080674       1        1  21.965056
19    -0.010897 -0.117647       2        1  18.807131
15    -0.101555 -0.103672       3        1  13.906959
2      0.000340 -0.113861       4        1  12.748866
3     -0.088771 -0.122338       5        1  12.003713
9      0.055305  0.208891       6        1   4.191191
10    -0.057588  0.053214       7        1   2.873688
0     -0.010489  0.020832       8        1   2.444238
4     -0.099204  0.009599       9        1   2.203599
16    -0.110405  0.002167      10        1   1.683462
11    -0.045550  0.065336      11        1   1.610515
6     -0.088027  0.056822      12        1   1.326942
5     -0.052381 -0.002139      13        1   1.231631
18    -0.070091 -0.081248      14        1   1.175142
12    -0.064196  0.029881      15        1   0.726471
8     -0.079979  0.082568      16        1   0.606111
1      0.029375  0.177331      17        1   0.273934
17     0.259038  0.105827      18        1   0.165112
14     0.290894 -0.095732      19        1   0.031815
7      0.282535 -0.095158      20        1   0.024424, topic_info=               Term         Freq        Total Category  logprob  loglift
292            city  3753.000000  3753.000000  Default  30.0000  30.0000
1728         church  1941.000000  1941.000000  Default  29.0000  29.0000
57           people  2998.000000  2998.000000  Default  28.0000  28.0000
20            death  2315.000000  2315.000000  Default  27.0000  27.0000
404            hand  1192.000000  1192.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
3570          mural     0.022300     0.953113  Topic20  -8.7968   4.5622
5550         seaman     0.022300     0.953113  Topic20  -8.7968   4.5622
2723  gratification     0.387450    24.688096  Topic20  -5.9418   4.1629
5862           poop     0.022300     0.953114  Topic20  -8.7968   4.5622
4316      emolument     0.022305    38.047462  Topic20  -8.7966   0.8756

[1472 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3642      1  0.280090  abdication
3642      3  0.430138  abdication
3642      4  0.090029  abdication
3642     12  0.190061  abdication
5094     10  0.890877    ablution
...     ...       ...         ...
1223     10  0.011136        zeal
1223     11  0.012250        zeal
1223     12  0.013364        zeal
1223     13  0.015591        zeal
1223     16  0.018932        zeal

[4463 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 20, 16, 3, 4, 10, 11, 1, 5, 17, 12, 7, 6, 19, 13, 9, 2, 18, 15, 8])

### Tf-idf topic model

In [48]:
# initialize a tfidf model
tfidf = models.TfidfModel(corpus)

In [49]:
# make a new corpus based on the tfidf model
corpus_tfidf = tfidf[corpus]

In [50]:
# here we build our topic model
lda_model_tfidf = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20, passes=50)
corpus_lda = lda_model_tfidf[corpus_tfidf]

In [51]:
lda_model_tfidf.print_topics()

[(0,
  '0.008*"baron" + 0.005*"bell" + 0.005*"pilgrim" + 0.005*"ruin" + 0.004*"temple" + 0.004*"column" + 0.003*"count" + 0.003*"rise" + 0.003*"work" + 0.003*"revolution"'),
 (1,
  '0.011*"pheasant" + 0.010*"sculptor" + 0.010*"lot" + 0.010*"dwell" + 0.010*"annalist" + 0.010*"absurdity" + 0.009*"tilt" + 0.008*"nakedness" + 0.007*"bull" + 0.007*"legend"'),
 (2,
  '0.012*"antiquary" + 0.012*"livery" + 0.011*"retrograde" + 0.010*"migrate" + 0.007*"methodize" + 0.007*"envelop" + 0.006*"aera" + 0.006*"wand" + 0.006*"anarchy" + 0.006*"observation"'),
 (3,
  '0.005*"pope" + 0.003*"church" + 0.003*"cardinal" + 0.003*"city" + 0.002*"senator" + 0.002*"people" + 0.002*"tribune" + 0.002*"bishop" + 0.002*"noble" + 0.002*"republic"'),
 (4,
  '0.014*"arch" + 0.012*"inscription" + 0.010*"kneeling" + 0.009*"amphitheatre" + 0.008*"enclosure" + 0.008*"abolition" + 0.007*"lodge" + 0.006*"embarrass" + 0.006*"mint" + 0.006*"elevate"'),
 (5,
  '0.022*"obelisk" + 0.015*"enumerate" + 0.010*"circulation" + 0.010

In [52]:
# Find topics in each document
lda_model_tfidf.get_document_topics(corpus_tfidf[0])

[(0, 0.10441803), (3, 0.7804112), (7, 0.024990667)]

In [53]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.068247  0.314195       1        1  72.598170
0      0.341962  0.042388       2        1  15.662391
18    -0.033844 -0.037845       3        1   1.898048
11     0.015460 -0.018593       4        1   0.952857
16     0.006123 -0.001390       5        1   0.705378
12    -0.004386 -0.026948       6        1   0.656757
15    -0.023504 -0.059752       7        1   0.653301
2     -0.012777 -0.025507       8        1   0.643565
7     -0.028811 -0.050788       9        1   0.638915
17    -0.023149 -0.022549      10        1   0.585721
19    -0.017226 -0.006504      11        1   0.578727
6     -0.027651 -0.028681      12        1   0.528342
1     -0.017677 -0.010715      13        1   0.521765
5     -0.014572 -0.009492      14        1   0.504216
4     -0.013338 -0.017525      15        1   0.503920
13    -0.022931 -0.011287      16        1   0.486045
14    -0.014185 -0.010094      17        1   0.478676
10    -0.014032 -0.006763      18        1   0.473195
8     -0.013171 -0.005414      19        1   0.465123
9     -0.014043 -0.006736      20        1   0.464888, topic_info=              Term      Freq      Total Category  logprob  loglift
5253      cardinal  53.00000  53.000000  Default  30.0000  30.0000
3953          pope  96.00000  96.000000  Default  29.0000  29.0000
398       election  27.00000  27.000000  Default  28.0000  28.0000
1728        church  50.00000  50.000000  Default  27.0000  27.0000
412           name  36.00000  36.000000  Default  26.0000  26.0000
...            ...       ...        ...      ...      ...      ...
17    constitution   0.01723   9.425026  Topic20  -8.7485  -0.9334
18          consul   0.01723  18.223059  Topic20  -8.7485  -1.5927
19         council   0.01723  19.609859  Topic20  -8.7485  -1.6660
20           death   0.01723  33.685751  Topic20  -8.7485  -2.2071
21         decline   0.01723  16.242724  Topic20  -8.7485  -1.4776

[919 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
0         1  0.956779       a.d.
4616      7  0.349273      abbot
4616      9  0.349273      abbot
1         1  0.967299    ability
2818      1  0.657844  abolition
...     ...       ...        ...
1166      1  0.750556     writer
1166      4  0.150111     writer
83        1  0.828814       year
83        2  0.138136       year
1223      1  0.968895       zeal

[879 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 19, 12, 17, 13, 16, 3, 8, 18, 20, 7, 2, 6, 5, 14, 15, 11, 9, 10])